1- OS precision

_The results show that: (1) Additive splitting is not stable; (2) among two stable methods, Strang splitting is more diffusive._
_The stability of Additive splitting deteriorates because the $-u_{sp}^n$ effectively changes the first terms of two Lax steps to second-order centered difference steps, changing its numerical stability. (Unstable with any CFL)_

_The extra dissipation of Strange splitting should be due to the smaller effective time steps. (?)_


_In addition, in each split step, stability requirement still needs to be satified and the overall time-step should be equal._

In [7]:
import sys 
import os   
sys.path.append(os.getcwd()+'/..')
import numpy as np
import math
import matplotlib.pyplot as plt 
import nm_lib as nm
from typing import Type
from matplotlib import animation
import matplotlib
matplotlib.use('TkAgg')

# --- Values reused in other tasks of ex_2 ---
x0 = -2.6 
xf = 2.6
pi = math.pi
a = -0.7
b = -0.3
# ---------------------------

def tfunc(xx):
    return math.cos(6.0 * pi * xx / 5.0) * math.cos(6.0 * pi * xx / 5.0) /  math.cosh(5.0 * xx * xx)

def discretisation(x0, xf, nump, dt_float: Type[float]):
    nint = nump - 1
    dh = dt_float((xf - x0) / nint)
    xx = dt_float(np.arange(nump) * dh + x0) 
    xx_half = dt_float(np.arange(nump) * dh + x0 + dh * 0.5)
    return nint, dh, xx, xx_half
    
nump = 257
nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

hh = np.zeros(nump)
for i in range(0, nump):
    hh[i] = tfunc(xx[i])

plt.figure() 
plt.plot(xx, hh, '-+', label='original function')
plt.xlabel("x")
plt.ylabel("u")
plt.legend(loc='lower right', shadow=False, fontsize='x-large')
plt.figtext(0.5, -0.1, "Original function with " + str(nump) + " grid points", horizontalalignment='center') 
plt.show()

In [8]:
def run_Burgers_Add(nump, nt, a, b, CFL, ddx = lambda x,y: nm.deriv_cent(x,y), bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc(xx[i])
 
    t, unnt = nm.ops_Lax_LL_Add(xx, hh, nt, a, b, CFL, ddx, 'wrap', bnd_limits)
        
    plt.figure() 
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt[:, 0].min() - 0.1 * (unnt[:, 0].max() - unnt[:, 0].min()),
        unnt[:, 0].max() + 0.1 * (unnt[:, 0].max() - unnt[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u")
 
    line, = plt.plot([], [], '--', label='Additive Splitting solution' + " CFL =" + str(CFL))
    plt.legend(loc='upper right')

    # Add time text
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)

    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line,

    def update(frame):
        line.set_data(xx, unnt[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line,

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    plt.show(block=True)

nump = 257  
#nt = 101
nt = 33
CFL = 0.4
a = 0.4
b = 0.6
#run_Burgers_Add(nump, nt, a, b, CFL)
#CFL = 0.9
#run_Burgers_Add(nump, nt, a, b, CFL)

In [9]:
def run_Burgers_Lie(nump, nt, a, b, CFL, ddx = lambda x,y: nm.deriv_cent(x,y), bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc(xx[i])
 
    t, unnt = nm.ops_Lax_LL_Lie(xx, hh, nt, a, b, CFL, ddx, 'wrap', bnd_limits)
        
    plt.figure() 
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt[:, 0].min() - 0.1 * (unnt[:, 0].max() - unnt[:, 0].min()),
        unnt[:, 0].max() + 0.1 * (unnt[:, 0].max() - unnt[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u")
 
    line, = plt.plot([], [], '--', label='Lie-Trotter Splitting solution' + " CFL =" + str(CFL))
    plt.legend(loc='upper right')

    # Add time text
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)

    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line,

    def update(frame):
        line.set_data(xx, unnt[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line,

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    plt.show(block=True)

nump = 257   
nt = 101
CFL = 0.4
a = 0.6
b = 0.4
#run_Burgers_Lie(nump, nt, a, b, CFL)
CFL = 0.9
#run_Burgers_Lie(nump, nt, a, b, CFL)

In [ ]:
def run_Burgers_Strang(nump, nt, a, b, CFL, ddx = lambda x,y: nm.deriv_cent(x,y), bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc(xx[i])
 
    t, unnt = nm.ops_Lax_LL_Strang(xx, hh, nt, a, b, CFL, ddx, 'wrap', bnd_limits)
        
    plt.figure() 
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt[:, 0].min() - 0.1 * (unnt[:, 0].max() - unnt[:, 0].min()),
        unnt[:, 0].max() + 0.1 * (unnt[:, 0].max() - unnt[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u")
 
    line, = plt.plot([], [], '--', label='Strang Splitting solution' + " CFL =" + str(CFL))
    plt.legend(loc='upper right')

    # Add time text
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)

    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line,

    def update(frame):
        line.set_data(xx, unnt[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line,

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    plt.show(block=True)

nump = 257   
nt = 101
CFL = 0.4
a = 0.6
b = 0.4
#run_Burgers_Strang(nump, nt, a, b, CFL)
#CFL = 0.9
#run_Burgers_Strang(nump, nt, a, b, CFL)

2- When does it not work? 

In [ ]:
# Strang splitting using Lax and Hyman methods

def run_Burgers_Strang_LH(nump, nt, a, b, CFL, ddx = lambda x,y: nm.deriv_cent(x,y), bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc(xx[i])
 
    t, unnt = nm.ops_Lax_LH_Strang(xx, hh, nt, a, b, CFL, ddx, 'wrap', bnd_limits)
        
    plt.figure() 
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt[:, 0].min() - 0.1 * (unnt[:, 0].max() - unnt[:, 0].min()),
        unnt[:, 0].max() + 0.1 * (unnt[:, 0].max() - unnt[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u")
 
    line, = plt.plot([], [], '--', label='Strang Lax-Hyman Splitting solution' + " CFL =" + str(CFL))
    plt.legend(loc='upper right')

    # Add time text
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)

    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line,

    def update(frame):
        line.set_data(xx, unnt[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line,

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    plt.show(block=True)

nump = 512   
nt = 100001
CFL = 0.8
a = 0.7
b = 0.3
run_Burgers_Strang_LH(nump, nt, a, b, CFL) 

In [ ]:
# Hyman method for the Burger's equation

def ops_Hyman_Add(xx: np.ndarray, hh: np.ndarray, nt: int, a: np.ndarray, b: np.ndarray, 
                  cfl_cut: float = 0.98, ddx=lambda x, y: nm.deriv_fwd(x, y), bnd_type: str = "wrap", bnd_limits: list = [0, 1], **kwargs,):
    if xx.size != hh.size:
        return None
    else:
        t = np.zeros(nt)
        unnt = np.zeros((xx.size, nt))
        unnt[:, 0] = hh
        fold = np.array([])
        dtold = 0.0
        for i in range(0, nt - 1):
            dt1 = nm.cfl_adv_burger(a, xx)
            dt2 = nm.cfl_adv_burger(b, xx)
            dt = min(dt1, dt2)
            #print(dt1, dt2, dt)
            du1 = np.zeros(hh.size)
            du2 = np.zeros(hh.size)
            du = np.zeros(hh.size)
            du1, fold, dtold = nm.hyman(xx, unnt[:, i], dt, a, fold = fold, dtold = dtold, cfl_cut = cfl_cut, ddx = ddx)
            du2, fold, dtold = nm.hyman(xx, unnt[:, i], dt, b, fold = fold, dtold = dtold, cfl_cut = cfl_cut, ddx = ddx)
            for j in range(1, hh.size-1):
                du[j] = du1[j] + du2[j] - unnt[j, i]
            du = np.pad(du[bnd_limits[0]:xx.size-bnd_limits[1]], bnd_limits, 'wrap')
            unnt[:, i+1] = du
            t[i+1] = t[i] + dt
        return t, unnt
    
def run_Burgers_Hyman_Add(nump, nt, a, b, CFL, ddx = lambda x,y: nm.deriv_cent(x,y), bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc(xx[i])
 
    t, unnt = ops_Hyman_Add(xx, hh, nt, a, b, CFL, ddx, 'wrap', bnd_limits)
        
    plt.figure() 
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt[:, 0].min() - 0.1 * (unnt[:, 0].max() - unnt[:, 0].min()),
        unnt[:, 0].max() + 0.1 * (unnt[:, 0].max() - unnt[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u")
 
    line, = plt.plot([], [], '--', label='Hyman-Additive Splitting solution' + " CFL =" + str(CFL) + " a =" + str(a) + " b =" + str(b))
    plt.legend(loc='upper right')

    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)

    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line,

    def update(frame):
        line.set_data(xx, unnt[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line,

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    plt.show(block=True)

nump = 512   
nt = 101
CFL = 0.4
a = 0.6
b = 0.4
run_Burgers_Hyman_Add(nump, nt, a, b, CFL) 

nt = 101
a = 1.0
b = 0.0
run_Burgers_Hyman_Add(nump, nt, a, b, CFL) 